# S3 CSV Processing

This notebook shows how to read CSV files from an S3 bucket. (note that the S3 bucket is public and doe not require credentials, so we use the AnonymousAWSCredentialsProvider)

The version of the org.apache.hadoop:hadoop-aws jar is dependent on the version included in the Bitnami Spark docker image.  If the notebook is unable to find the jar, open a terminal in the root level of the git repo and run `docker-compose exec spark-worker ls jars | grep hadoop-aws`.  This will print out the version of the hadoop-aws jar to use below.

In [1]:
import pyspark
import os

In [2]:
# S3_ACCESS_KEY = os.environ.get("S3_ACCESS_KEY")
# S3_SECRET_KEY = os.environ.get("S3_SECRET_KEY")
S3_ACCESS_KEY = "sparkaccesskey"
S3_SECRET_KEY = "sparksupersecretkey"
S3_ENDPOINT = "http://minio:9000"

In [3]:
conf = pyspark.SparkConf().setMaster("spark://spark:7077")
conf.set("spark.jars.packages", 'org.apache.hadoop:hadoop-aws:3.3.1')
# conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider')
conf.set('spark.hadoop.fs.s3a.endpoint', S3_ENDPOINT)
conf.set('spark.hadoop.fs.s3a.access.key', S3_ACCESS_KEY)
conf.set('spark.hadoop.fs.s3a.secret.key', S3_SECRET_KEY)
sc = pyspark.SparkContext(conf=conf)
sc.setLogLevel("INFO")

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4f50fab7-7af0-47ef-8a41-b90da8efa9a7;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.901 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 295ms :: artifacts dl 17ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.1 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	----------------------------------------------

22/09/05 03:13:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark = pyspark.sql.SparkSession(sc)

In [ ]:
df = spark.read.option("header", "true").csv("s3a://test/appl_stock.csv")

22/09/05 03:13:47 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
22/09/05 03:13:47 INFO SharedState: Warehouse path is 'file:/notebooks/spark-warehouse'.
22/09/05 03:13:48 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
22/09/05 03:13:48 INFO MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
22/09/05 03:13:48 INFO MetricsSystemImpl: s3a-file-system metrics system started
22/09/05 03:13:49 INFO CoarseGrainedSchedulerBackend$DriverEndpoint: Registered executor NettyRpcEndpointRef(spark-client://Executor) (172.18.0.4:35622) with ID 0,  ResourceProfileId 0
22/09/05 03:13:49 INFO BlockManagerMasterEndpoint: Registering block manager 172.18.0.4:42083 with 366.3 MiB RAM, BlockManagerId(0, 172.18.0.4, 42083, None)


In [ ]:
df.show()